In [3]:
from torch.autograd import Variable
import time
import copy
import torch
from torch import Tensor
import glob
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader

In [4]:
import matplotlib.pyplot as plt

In [5]:
class ECGDataset(Dataset):
    def __init__(
            self,
            npy_dir: os.PathLike,
            label_df : pd.DataFrame
    )-> None:
        self.npy_dir = npy_dir
        self.df = label_df

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self,index):
        file_name = self.df.iloc[index,0] 
   
        ecg_npy = np.load(os.path.join(self.npy_dir, file_name))
        ecg_tensor = torch.from_numpy(ecg_npy).float()
        
        return ecg_tensor , ecg_tensor
def load_data(train_npy_path,train_csv_path):
    total_df = pd.read_csv(train_csv_path, index_col=0)
    train_df, valid_df = train_test_split(total_df,test_size = 0.2, random_state = 42)
    
    train_set = ECGDataset(train_npy_path, train_df)
    valid_set = ECGDataset(train_npy_path, valid_df)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
    valid_loader =   DataLoader(valid_set, batch_size=32, shuffle=True, num_workers=0)
    
    return train_loader , valid_loader
    
def load_test_data(test_npy_path,test_csv_path):
    test_df = pd.read_csv(test_csv_path, index_col=0)
   
    test_set = ECGDataset(test_npy_path, test_df)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=True, num_workers=0)
    
    return test_loader

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.FC_1 = nn.Linear(150, 128)
        self.FC_2 = nn.Linear(128, 64)
        self.FC_3 = nn.Linear(64, 32)
        self.FC_4 = nn.Linear(32, 64)
        self.FC_5 = nn.Linear(64, 128)
        self.FC_6 = nn.Linear(128, 150)

    def forward(self, x):
        x = x.view(-1, 1 * 150)
        out = self.FC_1(x)
        out = F.relu(out)

        out = self.FC_2(out)
        out = F.relu(out)

        out = self.FC_3(out)
        out = F.relu(out)

        out = self.FC_4(out)
        out = F.relu(out)

        out = self.FC_5(out)
        out = F.relu(out)

        out = self.FC_6(out)


        return out
        
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()

        self.FC_1 = nn.Linear(150, 64)
        self.FC_2 = nn.Linear(64, 32)
        self.FC_3 = nn.Linear(32, 64)
        self.FC_4 = nn.Linear(64, 150)


    def forward(self, x):
        x = x.view(-1, 1 * 150)
        out = self.FC_1(x)
        out = F.relu(out)

        out = self.FC_2(out)
        out = F.relu(out)

        out = self.FC_3(out)
        out = F.relu(out)

        out = self.FC_4(out)
        return out

In [7]:
DEVICE = torch.device('cpu')  
PATH = "C:/Users/CSE-220425/Downloads/check_FC01.pt"

In [11]:
npy_path = "E:/01_AKI_tolopogy/01_AKI_tolopogy/hhj/AE_DATASET/TEST/ECG_AKI_N/"
csv_path = "E:/01_AKI_tolopogy/01_AKI_tolopogy/hhj/AE_DATASET/TEST/ECG_AKI_N_CSV/"
csv_list = os.listdir(csv_path)

In [6]:
AE = Net2().to(DEVICE)
# model.load_state_dict(torch.load(PATH, map_location=device))
AE.load_state_dict(torch.load(PATH, map_location=DEVICE ))
AE.eval()

Net2(
  (FC_1): Linear(in_features=150, out_features=64, bias=True)
  (FC_2): Linear(in_features=64, out_features=32, bias=True)
  (FC_3): Linear(in_features=32, out_features=64, bias=True)
  (FC_4): Linear(in_features=64, out_features=150, bias=True)
)

In [7]:
AE_loss = nn.MSELoss()

In [ ]:
df = pd.DataFrame()
for folder_id in filelist:
    test_loss = []
    npy_path ="D:/01_AKI_tolopogy/01_AKI_tolopogy/hhj/CONVAE_DATASET/TRAIN2/TRAIN/"
    csv_path =f"D:/01_AKI_tolopogy/01_AKI_tolopogy/hhj/CONVAE_DATASET/TEST2/AKI_N/{folder_id}"
    
    test_loader = load_test_data(npy_path,csv_path)
    for X, _ in test_loader:
        X = X.to(DEVICE)
        AE_output = AE(X)
        test_loss.append(AE_loss(AE_output, X).data)
    row = {"error":max(test_loss)}
    df.append(row,ignore_index=True)